# A Model for the Human Chromosome
This example shows a simple simulation of a single block copolymer that is a potential model of the Human Chromosome in hoomdd.

The human genome consists of 24 chromosomes with a total size of 3Gb.  Aiden et al. 2014 produced a 1kB resolution (below single gene resolution) Genome Contact Map.  Contact domains (∼185 kb) segregate into six nuclear subcompartments with distinct histone marks.
This can be reproduced phenomenologically with this simple block copolymer model.

from: https://lost-contact.mit.edu/afs//umich.edu/user/j/o/joaander/Public/hoomd-web/doc/page_example_scripts.html

## Initialize
First import hoomd and associated libraries and then initialize.  If no mode is given then the fastest processor cpu or gpu will be selected automatically.

In [1]:
#import hoomd
import hoomd.md
from hoomd import *
from hoomd.deprecated import *
context.initialize("");
#context.initialize("--mode=cpu");
#context.initialize("--mode=gpu");

HOOMD-blue v2.2.2-29-g1e86eec CUDA (8.0) DOUBLE HPMC_MIXED MPI SSE SSE2 
Compiled: 01/29/18
Copyright 2009-2017 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
notice(2): This system is not compute exclusive, using local rank to select GPUs
notice(2): Unable to identify node local rank information
notice(2): Using global rank to select GPUs
HOOMD-blue is running on the following GPU(s):
 [0]  GeForce GTX 1060 6GB  10 SM_6.1 @ 1.84 GHz, 6075 MiB DRAM, DIS


## Import other libraries and define parameters

The human genome has 24 chromasomes but for now we will simulate a single chromosome.

In [2]:
import math
# parameters
phi_P = 0.25
n_poly=1

## Define the polymer
The average length of a human chromosome is 133 Mb

In [3]:
import numpy as np
genome_size=3079843747
NChr=24 #Mb
Nbeads=genome_size/100000
#chromosome=np.linspace(1,Nchromosomes,Nchromosomes,dtype=intp).tolist()
chromosome_size=np.fix(np.r_[8,7.9,6.5,6.2,5.9,5.5,5.2,4.7,4.6,4.4,4.4,4.3,3.7,3.5,3.3,2.9,2.6,2.5,2.1,2.0,1.5,1.6,5.0,1.9]/100.0*Nbeads)
chromosome=list('cdefghijklmnopqurstuvwxy')
separation_radius='=0.35, '.join(chromosome[:NChr])+'=0.35'
print separation_radius
separation_radius=dict(c=0.35, d=0.35, e=0.35, f=0.35, g=0.35, h=0.35, i=0.35, j=0.35, k=0.35, l=0.35, m=0.35, n=0.35, o=0.35, p=0.35, q=0.35, u=0.35, r=0.35, s=0.35, t=0.35)
separation_radius.update(dict(u=0.35, v=0.35, w=0.35, x=0.35, y=0.35))

c=0.35, d=0.35, e=0.35, f=0.35, g=0.35, h=0.35, i=0.35, j=0.35, k=0.35, l=0.35, m=0.35, n=0.35, o=0.35, p=0.35, q=0.35, u=0.35, r=0.35, s=0.35, t=0.35, u=0.35, v=0.35, w=0.35, x=0.35, y=0.35


So if we assume that Euchromatin makes up 60% of the genome and heterochromatin makes up 40% then a block copolymer of 10000 beads would have a resolution of approximately 13 kb/bead the average size of a human gene.  This would correspond to approximately 18 beads per enhancer promoter loop.

In [4]:
#polymer1 = dict(bond_len=1.2, type=['A']*3000 + ['B']*4000 + ['A']*3000,bond="linear", count=n_poly)
polymer = [[] for i in range(NChr)]
for i in range(NChr): polymer[i] = dict(bond_len=1.2, type=chromosome[i]*int(chromosome_size[i]),bond="linear", count=1)
# perform some simple math to find the length of the box
#N = len(polymer1['type']) * polymer1['count']#+len(polymer2['type']) * polymer2['count']+len(polymer3['type']) * polymer3['count']
N=0
for i in range (NChr): N += len(polymer[i]['type']) * polymer[i]['count']#+len(polymer2['type']) * polymer2['count']+len(polymer3['type']) * polymer3['count']

# generate the polymer system
#init.create_random_polymers(box=data.boxdim(volume=10*math.pi * N / (6.0 * phi_P)), polymers=[polymer1,polymer2,polymer3],separation=separation_radius,seed=12)
init.create_random_polymers(box=data.boxdim(volume=5*math.pi * N / (6.0 * phi_P)), polymers=polymer,separation=separation_radius,seed=12)

notice(2): Group "all" created containing 30846 particles


## Setup the bonds and force fields

In [5]:
# force field setup
harmonic = hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('polymer', k=330.0, r0=0.84)
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(nlist=nl, r_cut=3.0)
dpd = hoomd.md.pair.dpd(r_cut=1.0, nlist=nl, kT=0.8, seed=1);
#dpd.pair_coeff.set('A', 'A', A=25.0, gamma = 4.5);
#dpd.pair_coeff.set('A', 'B', A=100.0, gamma = 4.5);
#dpd.pair_coeff.set('B', 'B', A=25.0, gamma = 4.5);
for i in range(NChr): 
    lj.pair_coeff.set(chromosome[i],chromosome[i], epsilon=1.0, sigma=1.0, alpha=1.0)
    #dpd.pair_coeff.set('A', 'A', A=25.0, gamma = 4.5);
    dpd.pair_coeff.set(chromosome[i],chromosome[i], A=0.0, gamma = 4.5);
for i in range(NChr):
    for j in range(i+1,NChr): 
        lj.pair_coeff.set(chromosome[i],chromosome[j], epsilon=1.0, sigma=1.0, alpha=0.0, r_cut=2**(1.0/6.0))
        #dpd.pair_coeff.set('A', 'B', A=100.0, gamma = 4.5);
        dpd.pair_coeff.set(chromosome[i], chromosome[j], A=0.0, gamma = 4.5);
#nl.reset_exclusions(exclusions = []);        

lj.set_params(mode='shift')

-----
You are using DPD. Please cite the following:
* C L Phillips, J A Anderson, and S C Glotzer. "Pseudo-random number generation
  for Brownian Dynamics and Dissipative Particle Dynamics simulations on GPU
  devices", Journal of Computational Physics 230 (2011) 7191--7201
-----


## Special Pairs will create our Contact Domains (not working yet)

In [6]:
#CTCF = hoomd.md.special_pair()
#lj_ctcf = hoomd.md.special_pair.lj(name="CTCF")
#lj_ctcf.pair_coeff.set('A','A', epsilon=1.0, sigma=1.0, r_cut=3)
#lj_ctcf.set_params(mode='shift')

## Integrate the simulation

In [7]:
all = group.all()
# integrate NVT for a bunch of time steps
#hoomd.md.integrate.mode_standard(dt=0.005)
#hoomd.md.integrate.nvt(group=all, kT=1.2, tau=0.5)
hoomd.md.integrate.mode_standard(dt=0.005)
hoomd.md.integrate.nve(group=all);
#hoomd.md.integrate.mode_standard(dt=0.005)
#hoomd.md.integrate.brownian(group=all, kT=1.2, seed=1, dscale=256)#10dt/k

The results of this example may be visualized with VMD which can be installed from here: http://www.ks.uiuc.edu/Research/vmd/

In [ ]:
hoomd.dump.gsd("gsd/24xchromosomes_1000_0.gsd", period=None, group=all, overwrite=True, time_step=0);
# setup the IMD server
hoomd.analyze.imd(port=54321, period=10)
# run a very long time so the simulation can be watched in VMD
run(1e6)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 48
notice(2): Particles with 2 exclusions             : 30798
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
notice(2): analyze.imd: listening on port 54321
Time 00:00:10 | Step 3536 / 1000000 | TPS 353.532 | ETA 00:46:58
notice(2): analyze.imd: accepted connection
Time 00:00:20 | Step 6921 / 1000000 | TPS 338.103 | ETA 00:48:57
Time 00:00:30 | Step 9957 / 1000000 | TPS 303.43 | ETA 00:54:22
Time 00:00:40 | Step 13014 / 1000000 | TPS 305.665 | ETA 00:53:48
Time 00:00:50 | Step 16021 / 1000000 | TPS 300.576 | ETA 00:54:33
Time 00:01:00 | Step 19091 / 1000000 | TPS 306.785 | ETA 00:53:17
Time 00:01:10 | Step 22209 / 1000000 | TPS 311.798 | ETA 00:52:15
Time 00:01:20 | Step 25257 / 1000000 | TPS 304.769 | ETA 00:53:18
Time 00:01:30 | Step 28360 / 1000000 | TPS 310.208 | ETA 00:52:12
Time 00:01:40 | Ste

Time 00:19:50 | Step 312291 / 1000000 | TPS 190.911 | ETA 01:00:02
Time 00:20:00 | Step 314058 / 1000000 | TPS 176.689 | ETA 01:04:42
Time 00:20:10 | Step 315881 / 1000000 | TPS 182.234 | ETA 01:02:34
Time 00:20:20 | Step 317614 / 1000000 | TPS 173.25 | ETA 01:05:38
Time 00:20:30 | Step 319375 / 1000000 | TPS 176.087 | ETA 01:04:25
Time 00:20:40 | Step 321105 / 1000000 | TPS 172.934 | ETA 01:05:25
Time 00:20:50 | Step 322818 / 1000000 | TPS 171.233 | ETA 01:05:54
Time 00:21:00 | Step 324521 / 1000000 | TPS 170.238 | ETA 01:06:07
Time 00:21:10 | Step 326212 / 1000000 | TPS 169.1 | ETA 01:06:24
Time 00:21:20 | Step 327858 / 1000000 | TPS 164.576 | ETA 01:08:04
Time 00:21:30 | Step 329506 / 1000000 | TPS 164.716 | ETA 01:07:50
Time 00:21:40 | Step 331142 / 1000000 | TPS 163.497 | ETA 01:08:10
Time 00:21:50 | Step 332783 / 1000000 | TPS 164.073 | ETA 01:07:46
Time 00:22:00 | Step 334421 / 1000000 | TPS 163.663 | ETA 01:07:46
Time 00:22:10 | Step 336063 / 1000000 | TPS 164.181 | ETA 01:07:2

Time 00:40:21 | Step 506701 / 1000000 | TPS 148.519 | ETA 00:55:21
Time 00:40:31 | Step 508179 / 1000000 | TPS 147.777 | ETA 00:55:28
Time 00:40:41 | Step 509676 / 1000000 | TPS 149.665 | ETA 00:54:36
Time 00:40:51 | Step 511180 / 1000000 | TPS 150.361 | ETA 00:54:10
Time 00:41:01 | Step 512691 / 1000000 | TPS 150.938 | ETA 00:53:48
Time 00:41:11 | Step 514185 / 1000000 | TPS 149.373 | ETA 00:54:12
Time 00:41:21 | Step 515684 / 1000000 | TPS 149.873 | ETA 00:53:51
Time 00:41:31 | Step 517181 / 1000000 | TPS 149.607 | ETA 00:53:47
Time 00:41:41 | Step 518682 / 1000000 | TPS 150.005 | ETA 00:53:28
Time 00:41:51 | Step 520171 / 1000000 | TPS 148.727 | ETA 00:53:46
Time 00:42:01 | Step 521682 / 1000000 | TPS 151.084 | ETA 00:52:45
Time 00:42:11 | Step 523174 / 1000000 | TPS 149.054 | ETA 00:53:19
Time 00:42:21 | Step 524670 / 1000000 | TPS 149.532 | ETA 00:52:58
Time 00:42:31 | Step 526163 / 1000000 | TPS 149.262 | ETA 00:52:54
Time 00:42:41 | Step 527623 / 1000000 | TPS 145.944 | ETA 00:5

The results of this example may be visualized with VMD which can be installed from here: http://www.ks.uiuc.edu/Research/vmd/

While running must execute the following command in a terminal:

In [ ]:
!cat imd.vmd
#vmd -e imd.vmd gsd/24xchromosomes_1000_0.gsd

![](snapshots/24xchromosomes_DPD.png)